# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd\__init__.py


# Testing various subscript Options


In [2]:
from __future__ import division 
import numpy as np 
from pysd import functions
from scipy.integrate import odeint
import itertools

### option 1: unnamed array

In [3]:
class MinModel(object):
    ##########  boilerplate stuff from the existing pysd #########
    def __init__(self):
        self._stocknames = [name[:-5] for name in dir(self) if name[-5:] == '_init']
        self._stocknames.sort() #inplace
        self._dfuncs = [getattr(self, 'd%s_dt'%name) for name in self._stocknames]
        self.state = dict(zip(self._stocknames, [None]*len(self._stocknames)))
        self.reset_state()
        self.functions = functions.Functions(self)

    def reset_state(self):
        """Sets the model state to the state described in the model file. """
        self.t = self.initial_time() #set the initial time
        retry_flag = False
        for key in self.state.keys():
            try:
                self.state[key] = eval('self.'+key+'_init()') #set the initial state
            except TypeError:
                retry_flag = True
        if retry_flag:
            self.reset_state() #potential for infinite loop!

    ########### Stuff we have to modify to make subscripts work #########
    def d_dt(self, state_vector, t):
        """The primary purpose of this function is to interact with the integrator.
        It takes a state vector, sets the state of the system based on that vector,
        and returns a derivative of the state vector
        """        
        self.set_state(state_vector)
        self.t = t
        
        derivative_vector = []
        for func in self._dfuncs:
            derivative_vector += list(func())
            
        return derivative_vector

    def set_state(self, state_vector):
        i = 0
        for key in self._stocknames:
            if isinstance(self.state[key], np.ndarray):
                size = self.state[key].size
                elements = state_vector[i:i+size]
                shape = self.state[key].shape
                self.state[key] = np.array(elements).reshape(shape)
                i += size
            else:
                self.state[key] = state_vector[i]
                i += 1

        
    def get_state(self):
        #if we keep this, we should make it fully a list comprehension
        state_vector = []
        for item in [self.state[key] for key in self._stocknames]:
            if isinstance(item, np.ndarray):
                state_vector += list(item.flatten())
            else:
                state_vector += list(item)
        return state_vector
    
    
    ######### model specific components (that go in the model file)
    suba_list = ['suba1', 'suba2', 'suba3']
    subb_list = ['suba2', 'subb2']
    
    def stock(self, suba, subb):
        return self.state['stock'][self.suba_list.index(suba)][self.subb_list.index(subb)] 
    
    def stock_init(self):
        return np.array([[1,1],[1,1],[1,1]])
    
    def dstock_dt(self):
        return [self.flow(suba, subb) for suba, subb in itertools.product(self.suba_list, self.subb_list)]
    
    def constant(self, suba, subb):
        values = np.array([[1,2],[3,4],[5,6]])
        return values[self.suba_list.index(suba)][self.subb_list.index(subb)] 
    
    def flow(self, suba, subb):
        return self.constant(suba, subb) * self.stock(suba, subb)
    
    def initial_time(self):
        return 0

In [4]:
a = MinModel()

In [5]:
a.dstock_dt()

[1, 2, 3, 4, 5, 6]

In [6]:
a.d_dt(a.get_state(), 0)

[1, 2, 3, 4, 5, 6]

In [7]:
odeint(a.d_dt, a.get_state(), range(10))

array([[  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  2.71828183e+00,   7.38905614e+00,   2.00855372e+01,
          5.45981515e+01,   1.48413168e+02,   4.03428857e+02],
       [  7.38905611e+00,   5.45981503e+01,   4.03428799e+02,
          2.98095808e+03,   2.20264678e+04,   1.62754835e+05],
       [  2.00855370e+01,   4.03428796e+02,   8.10308404e+03,
          1.62754798e+05,   3.26901777e+06,   6.56599942e+07],
       [  5.45981501e+01,   2.98095800e+03,   1.62754794e+05,
          8.88611090e+06,   4.85165268e+08,   2.64891352e+10],
       [  1.48413159e+02,   2.20264659e+04,   3.26901742e+06,
          4.85165219e+08,   7.20049123e+10,   1.06864811e+13],
       [  4.03428794e+02,   1.62754792e+05,   6.56599702e+07,
          2.64891236e+10,   1.06864768e+13,   4.31123465e+15],
       [  1.09663316e+03,   1.20260429e+06,   1.31881576e+09,
          1.44625715e+12,   1.58601383e+15,   1.73927639e+18],


###option 2: xray's `DataArray`

In [8]:
from xray import DataArray

class MinModel(object):
    ##########  boilerplate stuff from the existing pysd #########
    def __init__(self):
        self._stocknames = [name[:-5] for name in dir(self) if name[-5:] == '_init']
        self._stocknames.sort() #inplace
        self._dfuncs = [getattr(self, 'd%s_dt'%name) for name in self._stocknames]
        self.state = dict(zip(self._stocknames, [None]*len(self._stocknames)))
        self.reset_state()
        self.functions = functions.Functions(self)

    def reset_state(self):
        """Sets the model state to the state described in the model file. """
        self.t = self.initial_time() #set the initial time
        retry_flag = False
        for key in self.state.keys():
            try:
                self.state[key] = eval('self.'+key+'_init()') #set the initial state
            except TypeError:
                retry_flag = True
        if retry_flag:
            self.reset_state() #potential for infinite loop!

    ########### Stuff we have to modify to make subscripts work #########
    def d_dt(self, state_vector, t):
        """The primary purpose of this function is to interact with the integrator.
        It takes a state vector, sets the state of the system based on that vector,
        and returns a derivative of the state vector
        """        
        self.set_state(state_vector)
        self.t = t
        
        derivative_vector = []
        for func in self._dfuncs:
            derivative_vector += list(func())
            
        return derivative_vector

    def set_state(self, state_vector):
        i = 0
        for key in self._stocknames:
            if isinstance(self.state[key], DataArray):
                shape = self.state[key].shape
                size = self.state[key].size
                self.state[key].loc[:,:] = np.array(state_vector[i:i+size]).reshape(shape)
                i += size
            else:
                self.state[key] = state_vector[i]
                i += 1
        
    def get_state(self):
        #if we keep this, we should make it fully a list comprehension
        state_vector = []
        for item in [self.state[key] for key in self._stocknames]:
            if isinstance(item, DataArray):
                state_vector += list(item.values.flatten())
            else:
                state_vector += list(item)
        return state_vector
    
    
    ######### model specific components (that go in the model file)
    dim_dict = {'suba': ['subb1', 'subb2'],
                'subb': ['suba1', 'suba2', 'suba3']}
    
    def stock(self, suba, subb):
        return self.state['stock'].loc[suba, subb].values
    
    def stock_init(self):
        return DataArray([[1,1],[1,1],[1,1]], self.dim_dict)
    
    def dstock_dt(self):
        return [self.flow(suba, subb) for suba, subb in itertools.product(*self.dim_dict.values())]
    
    def constant(self, suba, subb):
        values = DataArray([[1,2],[3,4],[5,6]], self.dim_dict)
        return values.loc[suba, subb].values
    
    def flow(self, suba, subb):
        return self.constant(suba, subb) * self.stock(suba, subb)
    
    def initial_time(self):
        return 0

In [9]:
a = MinModel()

In [10]:
a.dstock_dt()

[1, 2, 3, 4, 5, 6]

In [11]:
a.get_state()

[1, 1, 1, 1, 1, 1]

In [12]:
a.d_dt(a.get_state(), 0)

[1, 2, 3, 4, 5, 6]

In [13]:
odeint(a.d_dt, a.get_state(), range(10))

C:\Anaconda\lib\site-packages\scipy\integrate\odepack.py:156: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ],
       [ 1.36660664,  1.73321329,  2.19963998,  2.93285319,  3.9990997 ,
         5.58153282],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])